In [20]:
import pandas as pd
import numpy as np
import os
import datetime

from hyperopt import Trials, STATUS_OK, tpe

from hyperas import optim
from hyperas.distributions import choice, uniform
from sklearn.preprocessing import MinMaxScaler

In [15]:
df = pd.read_csv("manipulated_pun.csv", sep=";", decimal=',', header='infer')

df.head()

,Data,Ora,PUN
0,20170101,1,53.30
1,20170101,2,52.00
2,20170101,3,51.00
3,20170101,4,47.27
4,20170101,5,45.49


In [16]:
x = df[['PUN']].values.astype(float)


In [21]:
min_max_scaler = MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)

df_normalized = pd.DataFrame(x_scaled)

df['PUN_n'] = df_normalized.values

In [22]:
df["Data"] = df["Data"].apply(lambda x: str(x))
df["Data"] = df["Data"].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d"))
df = df.loc[df['Data'] < '2020-01-01']

In [23]:
df["Ora2"] = pd.to_datetime(df.Ora, unit="h").dt.strftime("%H:%M")
df["Datetime"] = df["Data"].dt.strftime("%Y-%m-%d") + " " + df["Ora2"]
df.index = pd.DatetimeIndex(df.Datetime)

In [24]:
df.head()

,Data,Ora,PUN,PUN_n,Ora2,Datetime
Datetime,,,,,,
2017-01-01 01:00:00,2017-01-01,1,53.30,0.313529,01:00,2017-01-01 01:00
2017-01-01 02:00:00,2017-01-01,2,52.00,0.305882,02:00,2017-01-01 02:00
2017-01-01 03:00:00,2017-01-01,3,51.00,0.300000,03:00,2017-01-01 03:00
2017-01-01 04:00:00,2017-01-01,4,47.27,0.278059,04:00,2017-01-01 04:00
2017-01-01 05:00:00,2017-01-01,5,45.49,0.267588,05:00,2017-01-01 05:00


In [25]:
target = df["PUN_n"]
df["df24"] = target.shift(168)
target = df["PUN_n"].iloc[169:]
features = df[["Ora","PUN_n", "df24"]].iloc[169:]
features

,Ora,PUN_n,df24
Datetime,,,
2017-01-08 02:00:00,2,0.304824,0.305882
2017-01-08 03:00:00,3,0.301471,0.300000
2017-01-08 04:00:00,4,0.293941,0.278059
2017-01-08 05:00:00,5,0.282294,0.267588
2017-01-08 06:00:00,6,0.282353,0.268765
...,...,...,...
2019-12-31 20:00:00,20,0.332235,0.279000
2019-12-31 21:00:00,21,0.313176,0.268824
2019-12-31 22:00:00,22,0.298118,0.214882


In [ ]:
features.to_csv(r'file2.csv', index = False)

In [41]:
df = pd.read_csv("file2.csv", sep=",", decimal='.', header='infer')
df

,Ora,PUN_n,df24
0,2,0.304824,0.305882
1,3,0.301471,0.300000
2,4,0.293941,0.278059
3,5,0.282294,0.267588
4,6,0.282353,0.268765
...,...,...,...
26106,20,0.332235,0.279000
26107,21,0.313176,0.268824
26108,22,0.298118,0.214882
26109,23,0.285941,0.161529


In [73]:
def data():
    from sklearn.model_selection import train_test_split
    df = pd.read_csv("file2.csv", sep=",", decimal='.', header='infer')
    train, test = train_test_split(df, test_size=0.2)
    df = df.drop(['df24'], axis =1)
    X_train = train[["df24"]][168:]
    X_test = test[["df24"]]
    y_train = train[["PUN_n"]][168:]
    y_test = test[["PUN_n"]]
    X_train = X_train.values
    X_test = X_test.values
    return X_train, y_train, X_test, y_test


In [78]:
def create_model(X_train, y_train, X_test, y_test):
    import tensorflow.python.keras
    from tensorflow.python.keras.models import Sequential
    from tensorflow.python.keras.layers import Dense, Activation, Dropout
    from tensorflow.python.keras import backend as K

    from tensorflow.python.keras import utils, layers, Sequential
    
    model = Sequential()
    # the model will take as input arrays of shape (*, X_train.shape[1])
    # and output arrays of shape (*, 25)
    model.add(Dense(25, input_shape=(X_train.shape[1],)))
    model.add(Activation('relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([64, 128, 256, 512, 1024])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, 1)}}))

    model.add(Dense(1))
    model.add(Activation('softmax'))

    model.compile(loss='mse', metrics=['mean_absolute_error'],
                  optimizer={{choice(['rmsprop', 'adam', 'sgd'])}})

    result = model.fit(X_train, y_train,
              batch_size={{choice([22, 64, 128, 256, 512, 1024])}},
              epochs=1000,
              verbose=2,
              validation_split=0.1)
    #get the highest validation mae of the training epochs
    validation_mae = np.amax(result.history['mean_absolute_error']) 
    print('Best validation mae of epoch:', validation_mae)
    return {'loss': validation_mae, 'status': STATUS_OK, 'model': model}


In [79]:
best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=5,
                                      trials=Trials(),
                                      notebook_name='Ora')

>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import os
except:
    pass

try:
    import datetime
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from sklearn.preprocessing import MinMaxScaler
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    import tensorflow.python.keras
except:
    pass

try:
    from tensorflow.python.keras.models import Sequential
except:
    pass

try:
    from tensorflow.python.keras.layers import Dense, Activation, Dropout
except:
    pass

try:
    from tensorflow.python.keras import backend as K
except:
    pass

try:
    from tensorflow.python.keras import utils, layers, Sequential
except:
    pass

>>> Hyperas search space:

def get_space():

 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 33/1000                                        
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 34/1000                                        
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 35/1000                                        
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 36/1000                                        
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 37/1000                                        
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 38/1000                                        
 - 1s - loss: 0.4591 - mean_absolute_error: 0.

Epoch 84/1000                                        
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 85/1000                                        
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 86/1000                                        
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 87/1000                                        
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 88/1000                                        
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 89/1000                                        
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 90/1000                                 

 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 136/1000                                       
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 137/1000                                       
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 138/1000                                       
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 139/1000                                       
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 140/1000                                       
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 141/1000                                       
 - 1s - loss: 0.4591 - mean_absolute_error: 0.

Epoch 187/1000                                       
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 188/1000                                       
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 189/1000                                       
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 190/1000                                       
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 191/1000                                       
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 192/1000                                       
 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 193/1000                                

 - 1s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 239/1000                                       
 - 2s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 240/1000                                       
 - 2s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 241/1000                                       
 - 2s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 242/1000                                       
 - 2s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 243/1000                                       
 - 2s - loss: 0.4591 - mean_absolute_error: 0.6716 - val_loss: 0.4600 - val_mean_absolute_error: 0.6725

Epoch 244/1000                                       
 - 2s - loss: 0.4591 - mean_absolute_error: 0.

KeyboardInterrupt: 

In [77]:
print("Best performing model chosen hyper-parameters:")
print(best_run)

Best performing model chosen hyper-parameters:
{'Activation': 0, 'Dense': 4, 'Dropout': 0.8444244099007299, 'Dropout_1': 0.5350807190884803, 'batch_size': 5, 'optimizer': 1}
